# Logistic Regression

In [18]:
import matplotlib.pyplot as plt
import pandas as pd
import statsmodels.api as sm
import numpy as np

In [3]:
# using NHANES data
da = pd.read_csv("nhanes_2015_2016.csv")

# 실습에 사용할 columns만 추출
# BPXSY1: systolic blood pressure(혈압), RIDAGEYR: 나이, RIAGENDR: 성별(1:남성, 2:여성)
# DMDEDUC2: 교육 수준
# SMQ020: 흡연 유무(1: 흡연, 2: 비흡연, 7: 모름, 9: 답변 거부)
vars = ["BPXSY1", "RIDAGEYR", "RIAGENDR", "DMDEDUC2", "BMXBMI", "SMQ020"]
da = da[vars].dropna()
da.head()

,BPXSY1,RIDAGEYR,RIAGENDR,DMDEDUC2,BMXBMI,SMQ020
0,128.0,62,1,5.0,27.8,1
1,146.0,53,1,3.0,30.8,1
2,138.0,78,1,3.0,28.8,1
3,132.0,56,2,5.0,42.4,2
4,100.0,42,2,4.0,20.3,2


## logistic regression을 통해   
## 독립변수들(나이, 성별, 교육 수준 등)이 종속변수(흡연 유무)에 미치는 영향 확인

In [6]:
da["RIAGENDRx"] = da.RIAGENDR.replace({1: "Male", 2: "Female"})

In [9]:
# SMQ020: 7, 9(모름, 답변거부) = missing value
da["smq"] = da.SMQ020.replace({2: 0, 7: np.nan, 9: np.nan})

# smq 1: 흡연, 0: 비흡연

## 1) odds and log odds
   
Linear regression과 달리 logistic regression은 종속변수가 binary value이다.   
따라서 y가 1이 될 확률 p와, 0이 될 확률 1-p로 

odds = p/(1-p)

In [11]:
c = pd.crosstab(da.RIAGENDRx, da.smq).apply(lambda x: x/x.sum(), axis=1)
c["odds"] = c.loc[:, 1] / c.loc[:, 0]
c

smq,0.0,1.0,odds
RIAGENDRx,,,
Female,0.680197,0.319803,0.470162
Male,0.467453,0.532547,1.139252


In [12]:
c["logodds"] = np.log(c.odds)
c

smq,0.0,1.0,odds,logodds
RIAGENDRx,,,,
Female,0.680197,0.319803,0.470162,-0.754679
Male,0.467453,0.532547,1.139252,0.130371


## 2) logistic regression

In [19]:
model = sm.GLM.from_formula("smq ~ RIAGENDRx", family=sm.families.Binomial(), data=da)
result = model.fit()
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:                    smq   No. Observations:                 5094
Model:                            GLM   Df Residuals:                     5092
Model Family:                Binomial   Df Model:                            1
Link Function:                  logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -3350.6
Date:                Mon, 09 Aug 2021   Deviance:                       6701.2
Time:                        16:11:10   Pearson chi2:                 5.09e+03
No. Iterations:                     4                                         
Covariance Type:            nonrobust                                         
=====================================================================================
                        coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------
Intercept            -0.7547      0.042    -18.071      0.000      -0.837      -0.673
RIAGENDRx[T.Male]     0.8851      0.058     15.227      0.000       0.771       0.999
=====================================================================================
"""

In [20]:
c.logodds.Male - c.logodds.Female

0.8850500036644218

## 해석

DV(종속변수): BPXSY1    /    IV(독립변수) : RIDAGEYR   
나이가 1살 더 많아질수록, SBP(혈압)가 **평균적으로** 0.48 단위 증가한다.   
p값은 거의 0이기 때문에 유의수준 0.05에서 유의하다.   


#### R-square
R-square 값은 0.207 : 약 21%의 SBP가 age로 인해 설명된다.   
age만으로는 설명되지 않는 부분이 많다.   
age뿐만 아니라 다른 변수들(성별, BMI수치 등)에 의해 영향받는 부분이 있을 것이고, 이 변수들을 모델에 추가하면 R-square가 증가할 것이다.   
   
   
독립변수가 하나일때, R-square 값은 dv와 iv의 상관계수의 제곱과 같다.   

In [4]:
da[["BPXSY1", "RIDAGEYR"]].corr()

,BPXSY1,RIDAGEYR
BPXSY1,1.000000,0.455142
RIDAGEYR,0.455142,1.000000


In [5]:
correff = da[["BPXSY1", "RIDAGEYR"]].corr()
correff**2

,BPXSY1,RIDAGEYR
BPXSY1,1.000000,0.207155
RIDAGEYR,0.207155,1.000000


# 

## 3) Adding covariates   
   
'RIAGENDR' : 성별(1:남성, 2:여성) 변수 추가   
성별 변수는 categorical 변수이다.

In [21]:
da["RIAGENDRx"] = da.RIAGENDR.replace({1: "Male", 2: "Female"})

In [22]:
model = sm.GLM.from_formula("smq ~ RIDAGEYR + RIAGENDRx", family=sm.families.Binomial(), data=da)
result = model.fit()
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:                    smq   No. Observations:                 5094
Model:                            GLM   Df Residuals:                     5091
Model Family:                Binomial   Df Model:                            2
Link Function:                  logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -3296.6
Date:                Mon, 09 Aug 2021   Deviance:                       6593.2
Time:                        16:43:24   Pearson chi2:                 5.10e+03
No. Iterations:                     4                                         
Covariance Type:            nonrobust                                         
=====================================================================================
                        coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------
Intercept            -1.6166      0.095    -16.985      0.000      -1.803      -1.430
RIAGENDRx[T.Male]     0.8920      0.059     15.170      0.000       0.777       1.007
RIDAGEYR              0.0172      0.002     10.289      0.000       0.014       0.021
=====================================================================================
"""

## 해석

1. 같은 성별일 때, age가 1 높으면 BPXSY1이 평균적으로 0.47 더 높다.
2. 같은 나이일 때, 남성일 때 BPXSY1이 평균적으로 3.23 더 높다.
    
   
하나의 회귀 계수를 해석할 때, 다른 변수는 모두 동일하다.   
age 회귀 계수를 해석할 때, 같은 성별 기준으로 해석한다.   
성별 회귀 계수를 해석할 때, 같은 나이 기준으로 해석한다.

   
   
EX) 50살 남자와 40살 여자의 혈압을 비교해보면,    
위 모델에 따라 남자가 혈압이 3.23 + 10 * (0.47) = 7.93 단위 더 높다.

#### categorical 변수 해석
categorical 변수는 모델 내에서 **'더미 변수'** 로 활용된다.   
categorical 변수에서 기준이 되는 값(여성)과 비교해 다른 값(남성)들의 결과를 비교한다.   
기준이 되는 여성은 결과표에 나타나지 않는다.   
RIAGENDRx[T.Male]	3.2322 은 기준이 되는 여성에 비해 남성일 경우, 종속변수가 3.23 더 높다라는 것을 의미한다.

#### RIDAGEYR 회귀 계수
   
이전 모델과 비교했을 때, age의 계수가 조금 증가하였다.(거의 변화가 없다.)    
이것은 새로 추가한 변수(성별)이 age와 상관관계가 거의 없음을 의미한다.   


In [8]:
da[["RIDAGEYR", "RIAGENDR"]].corr()

,RIDAGEYR,RIAGENDR
RIDAGEYR,1.000000,-0.021398
RIAGENDR,-0.021398,1.000000


# 

## 3) three predictors

DMDEDUC2(교육 수준) 변수 추가   
1: less than 9th grade   
2: 9-11th grade   
3: high school   
4: some college   
5: college   
7: 답변 거부   
9: 모른다   

In [25]:
da["DMDEDUC2x"] = da.DMDEDUC2.replace({1: "lt9", 2: "x9_11", 3: "HS", 4: "SomeCollege",
                                       5: "College", 7: np.nan, 9: np.nan})

In [26]:
model = sm.GLM.from_formula("smq ~ RIDAGEYR + RIAGENDRx + DMDEDUC2x", family=sm.families.Binomial(), data=da)
result = model.fit()
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:                    smq   No. Observations:                 5093
Model:                            GLM   Df Residuals:                     5086
Model Family:                Binomial   Df Model:                            6
Link Function:                  logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -3201.2
Date:                Mon, 09 Aug 2021   Deviance:                       6402.4
Time:                        16:47:13   Pearson chi2:                 5.10e+03
No. Iterations:                     4                                         
Covariance Type:            nonrobust                                         
============================================================================================
                               coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------------
Intercept                   -2.3060      0.114    -20.174      0.000      -2.530      -2.082
RIAGENDRx[T.Male]            0.9096      0.060     15.118      0.000       0.792       1.028
DMDEDUC2x[T.HS]              0.9434      0.090     10.521      0.000       0.768       1.119
DMDEDUC2x[T.SomeCollege]     0.8322      0.084      9.865      0.000       0.667       0.998
DMDEDUC2x[T.lt9]             0.2662      0.109      2.438      0.015       0.052       0.480
DMDEDUC2x[T.x9_11]           1.0986      0.107     10.296      0.000       0.889       1.308
RIDAGEYR                     0.0183      0.002     10.582      0.000       0.015       0.022
============================================================================================
"""

## 해석

1. 같은 성별, 같은 BMI일 때, 나이가 1 높으면 BPXSY1이 평균적으로 0.47 더 높다.
2. 같은 나이, 같은 BMI일 때, 남성일 때 BPXSY1이 평균적으로 3.58 더 높다.
3. 같은 성별, 같은 나이일 때, BMI가 1 단위 높으면 BPXSY1이 평균적으로 0.3 더 높다.

#### 회귀 계수 변화
이전 모델과 비교했을 때,   
age 계수는 0.4739에서 0.4709로   
성별 계수는 3.23에서 3.58로 변화하였다.   

BMI 변수를 추가하고나서 회귀 계수의 변화량을 보아 BMI 변수는 나이보다 성별과 더 상관관계가 있을 것 같다.

In [10]:
da[["RIDAGEYR", "RIAGENDR", "BMXBMI"]].corr()

,RIDAGEYR,RIAGENDR,BMXBMI
RIDAGEYR,1.000000,-0.021398,0.023089
RIAGENDR,-0.021398,1.000000,0.080463
BMXBMI,0.023089,0.080463,1.000000
